In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from pycaret.regression import *
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, r2_score
df_NACA5 = pd.read_csv("data/df_NACA5.csv")
df_NACA5.head(5)

In [ ]:
def split_data(df):
    df = df.sample(frac = 1, random_state = 0)
    train  = df.head(0)
    test = df.head(0)
    for ID in set(df['Airfoil_ID']):
        df_tmp = df[df['Airfoil_ID']==ID]
        N = int(len(df_tmp)*0.9)
        train = pd.concat([train,df_tmp.iloc[:N]])
        test = pd.concat([test,df_tmp.iloc[N:]])
    train_df = train.reset_index(drop=True)
    test_df = test.reset_index(drop=True)
    return train_df, test_df

In [ ]:
train_df, test_df = split_data(df_NACA5)
train_df.head(5)

In [ ]:
test_df.head(5)

In [ ]:
def compare(df, use_cols, target):
    exp = setup(
        data = df[use_cols + [target]],
        target = target,
        numeric_features = use_cols,
        session_id = 42,
        normalize = True, 
        transformation = False,   
        verbose = False
        )
    compare_models()

In [ ]:
def predict(train_df, test_df, use_cols, target, model_name):
    exp = setup(
        data=train_df[use_cols + [target]],
        target=target,
        numeric_features=use_cols,
        session_id=42,
        normalize=True, 
        transformation=False,   
        verbose=False
        )
    model = create_model(model_name)
    plot_model(model, plot='error')
    final_model = finalize_model(model)
    plot_model(model, plot='feature')
    result = predict_model(final_model, data = test_df[use_cols])
    return result["prediction_label"]

In [ ]:
target = 'Drag'
use_cols = ['L', 'P', 'Q', 'XX', 'Re', 'AOA']
compare(df_NACA5, use_cols, target)

In [ ]:
model_name = 'et'
y_pred = predict(train_df, test_df, use_cols, target, model_name)
y_true = test_df[target]
print("MAE =", mean_absolute_error(y_true, y_pred))
print("R2  =", r2_score(y_true, y_pred))

In [ ]:
target = 'Lift'
compare(df_NACA5, use_cols, target)

In [ ]:
model_name = 'et'
y_pred = predict(train_df, test_df, use_cols, target, model_name)
y_true = test_df[target]
print("MAE =", mean_absolute_error(y_true, y_pred))
print("R2  =", r2_score(y_true, y_pred))